<a href="https://colab.research.google.com/github/ailtiakova/awesome-for-beginners/blob/master/RFM_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#RFM Analyis 

The purpose of this exercise is to create the RFM metrics, in order to proceed with Customer Segmentation. This is a form of behavioural segmentation. 

**RFM** is an acronym of recency, frequency and monetary. 


Source:https://learn.datacamp.com/courses/customer-segmentation-in-python

https://github.com/jjone36/Cohort/blob/master/Cohort_Anaylsis_Medium.ipynb

In [0]:
import pandas as pd
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
df1 = pd.read_excel(url)
# Dataset is now stored in a Pandas Dataframe

In [0]:
# drop the row missing customer ID 
df1 = df1[df1.CustomerID.notnull()]
df1 = df1.sample(frac = .3).reset_index(drop = True)
# Shuffle your dataframe in-place and reset the index by specifying drop=True, this prevents from creating a column containing the old index entries.
# The frac keyword argument specifies the fraction of rows to return in the random sample, so frac=1 means return all rows (in random order).


In [11]:
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,557328,21481,FAWN BLUE HOT WATER BOTTLE,1,2011-06-20 09:53:00,2.95,14646.0,Netherlands
1,537334,85135B,BLUE DRAGONFLY HELICOPTER,1,2010-12-06 12:06:00,7.95,16719.0,United Kingdom
2,543731,21121,SET/10 RED POLKADOT PARTY CANDLES,24,2011-02-11 11:57:00,1.25,17677.0,United Kingdom
3,560716,22467,GUMBALL COAT RACK,6,2011-07-20 13:26:00,2.55,17675.0,United Kingdom
4,543639,84459A,PINK METAL CHICKEN HEART,1,2011-02-10 16:52:00,1.49,16725.0,United Kingdom


Invoice dates are no longer in chronological order.

In [0]:
import datetime as dt
# extract year, month and day
df1['InvoiceDay'] = df1.InvoiceDate.apply(lambda x: dt.datetime(x.year, x.month, x.day))
# lambda functions is a small anonymous function. A lambda function can take any number of arguments, but can only have one expression.

In [22]:
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceDay
0,557328,21481,FAWN BLUE HOT WATER BOTTLE,1,2011-06-20 09:53:00,2.95,14646.0,Netherlands,2011-06-20
1,537334,85135B,BLUE DRAGONFLY HELICOPTER,1,2010-12-06 12:06:00,7.95,16719.0,United Kingdom,2010-12-06
2,543731,21121,SET/10 RED POLKADOT PARTY CANDLES,24,2011-02-11 11:57:00,1.25,17677.0,United Kingdom,2011-02-11
3,560716,22467,GUMBALL COAT RACK,6,2011-07-20 13:26:00,2.55,17675.0,United Kingdom,2011-07-20
4,543639,84459A,PINK METAL CHICKEN HEART,1,2011-02-10 16:52:00,1.49,16725.0,United Kingdom,2011-02-10


In [24]:
# print the time period
print('Min : {}, Max : {}'.format(min(df1.InvoiceDay), max(df1.InvoiceDay)))

Min : 2010-12-01 00:00:00, Max : 2011-12-09 00:00:00


In [0]:
# pin the last date
pin_date = max(df1.InvoiceDay) + dt.timedelta(1)

In [27]:
# Create total spend dataframe
df1['TotalSum'] = df1.Quantity * df1.UnitPrice
# you can create metrics from existing columns
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceDay,TotalSum
0,557328,21481,FAWN BLUE HOT WATER BOTTLE,1,2011-06-20 09:53:00,2.95,14646.0,Netherlands,2011-06-20,2.95
1,537334,85135B,BLUE DRAGONFLY HELICOPTER,1,2010-12-06 12:06:00,7.95,16719.0,United Kingdom,2010-12-06,7.95
2,543731,21121,SET/10 RED POLKADOT PARTY CANDLES,24,2011-02-11 11:57:00,1.25,17677.0,United Kingdom,2011-02-11,30.00
3,560716,22467,GUMBALL COAT RACK,6,2011-07-20 13:26:00,2.55,17675.0,United Kingdom,2011-07-20,15.30
4,543639,84459A,PINK METAL CHICKEN HEART,1,2011-02-10 16:52:00,1.49,16725.0,United Kingdom,2011-02-10,1.49


In [0]:
# calculate RFM values, aggregated for each Customer using the Groupby function.
rfm = df1.groupby('CustomerID').agg({
    'InvoiceDate' : lambda x: (pin_date - x.max()).days,  
    'InvoiceNo' : 'count', #works out numer of discrete purchases by the customer, ie. is this first, second, sixty-sixth purchase?
    'TotalSum' : 'sum'})



In [29]:
# rename the columns.
#This method is quite useful when we need to rename some selected columns because we need to specify information only for the columns which are to be renamed.
rfm.rename(columns = {'InvoiceDate' : 'Recency',
                      'InvoiceNo' : 'Frequency', 
                      'TotalSum' : 'Monetary'}, inplace = True)
rfm.head()

,Recency,Frequency,Monetary
CustomerID,,,
12347.0,2,66,1540.21
12348.0,75,13,630.84
12349.0,18,30,549.48
12350.0,310,6,111.70
12352.0,36,30,869.08


In [0]:
# create labels and assign them to tree percentile groups 
r_labels = range(4, 0, -1)
r_groups = pd.qcut(rfm.Recency, q = 4, labels = r_labels)
# r_labels, in descending order. Recency means how much time has elapsed since a customer’s last order.
# The smaller the value is, the more engaged a customer to that brand. 
f_labels = range(1, 5)
f_groups = pd.qcut(rfm.Frequency, q = 4, labels = f_labels)
m_labels = range(1, 5)
m_groups = pd.qcut(rfm.Monetary, q = 4, labels = m_labels)

In [32]:
# make a new column for group labels
rfm['R'] = r_groups.values
rfm['F'] = f_groups.values
rfm['M'] = m_groups.values
# sum up the three columns
rfm['RFM_Segment'] = rfm.apply(lambda x: str(x['R']) + str(x['F']) + str(x['M']), axis = 1)
# Axis along which the function is applied: 1 or ‘columns’: apply function to each row.
rfm['RFM_Score'] = rfm[['R', 'F', 'M']].sum(axis = 1)
rfm.head()

,Recency,Frequency,Monetary,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12347.0,2,66,1540.21,4,4,4,444,12.0
12348.0,75,13,630.84,2,2,4,224,8.0
12349.0,18,30,549.48,3,3,4,334,10.0
12350.0,310,6,111.70,1,1,2,112,4.0
12352.0,36,30,869.08,3,3,4,334,10.0


In [33]:
# calculate average values for each RFM
rfm_agg = rfm.groupby('RFM_Score').agg({
    'Recency' : 'mean',
    'Frequency' : 'mean',
    'Monetary' : ['mean', 'count']
})
rfm_agg.round(1).head()

Recency Frequency Monetary      
             mean      mean     mean count
RFM_Score                                 
3.0         256.9       2.9     35.7   380
4.0         177.3       4.6     70.7   408
5.0         138.3       6.7    103.0   471
6.0         103.2       9.2    169.0   461
7.0          79.9      13.3    219.3   437

In [34]:
# assign labels from total score
score_labels = ['Green', 'Bronze', 'Silver', 'Gold']
score_groups = pd.qcut(rfm.RFM_Score, q = 4, labels = score_labels)
rfm['RFM_Level'] = score_groups.values
rfm.head()

,Recency,Frequency,Monetary,R,F,M,RFM_Segment,RFM_Score,RFM_Level
CustomerID,,,,,,,,,
12347.0,2,66,1540.21,4,4,4,444,12.0,Gold
12348.0,75,13,630.84,2,2,4,224,8.0,Silver
12349.0,18,30,549.48,3,3,4,334,10.0,Silver
12350.0,310,6,111.70,1,1,2,112,4.0,Green
12352.0,36,30,869.08,3,3,4,334,10.0,Silver
